# Chassis service

Import libraries that have been installed before

In [1]:
import chassisml
import sklearn
import mlflow.pyfunc
from joblib import dump, load

### Train the model

This will train a sklearn model and it will be saved as a joblib file inside the `model` directory.

The goal for Chassis service is to create an image that exposes this model.

In [2]:
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)
dump(clf, './model.joblib')

['./model.joblib']

In [3]:
# Wrap your model in a pyfunc and provide auxiliary functionality through extension of the
# mlflow PythonModel class with methods pre_process, post_process, and explain

class CustomModel(mlflow.pyfunc.PythonModel):
    _model = load('./model.joblib')
    
    def load_context(self, context):
        self.model = self._model

    def predict(self, context, inputs):
        processed_inputs = self.pre_process(inputs)
        inference_results = self.model.predict(processed_inputs)
        return self.post_process(inference_results)

    def pre_process(self, inputs):
        return inputs / 2

    def post_process(self, inference_results):
        structured_results = []
        for inference_result in inference_results:
            inference_result = {
                "classPredictions": [
                    {"class": str(inference_result), "score": str(1)}
                ]
            }
            structured_output = {
                "data": {
                    "result": inference_result,
                    "explanation": None,
                    "drift": None,
                }
            }
            structured_results.append(structured_output)
        return structured_results

    def explain(self, images):
        pass

In [4]:
# Define conda environment with all required dependencies for your model

conda_env = {
    "channels": ["defaults", "conda-forge", "pytorch"],
    "dependencies": [
        "python=3.8.5",
        "pytorch",
        "torchvision",
        "pip",
        {
            "pip": [
                "mlflow",
                "lime",
                "sklearn"
            ],
        },
    ],
    "name": "linear_env"
}

### Train the model

Transform the model into MLFlow format.

In [5]:
model_save_path = "mlflow_custom_pyfunc_svm"
mlflow.pyfunc.save_model(path=model_save_path, python_model=CustomModel(), conda_env=conda_env)

Load the MLFlow model and test it.

In [6]:
import json

classifier = mlflow.pyfunc.load_model(model_save_path)
predictions = classifier.predict(X_test)
print(json.dumps(predictions[0], indent=4))

{
    "data": {
        "result": {
            "classPredictions": [
                {
                    "class": "8",
                    "score": "1"
                }
            ]
        },
        "explanation": null,
        "drift": null
    }
}


We check that the model has been correctly saved inside the `model` directory.

In [7]:
!ls ./mlflow_custom_pyfunc_svm

conda.yaml  MLmodel  python_model.pkl


### Define the values needed

Since now we are just creating and downloading the docker image, the only fields that Chassis service actually needs are:

* `model_name`: name for the model inside the image
* `model_path`: directory that contains our model file

In [8]:
# !echo -n "<user>:<password>" | base64 # XXXX:XXXX -> XxXxXxXx
!echo -n "XXXX:XXXX" | base64

XxXxXxXx


In [9]:
image_data = {
    'name': 'XXXX/chassisml-sklearn-demo:latest',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': './mlflow_custom_pyfunc_svm',
    'registry_auth': 'XxXxXxXx'
}

### Forward ports to access service and registry

This assumes that you are running these commands on your own terminal to redirect the service (port 5000) and the registry (port 5001) to localhost.

In [ ]:
! # kubectl port-forward service/chassis 5000:5000

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: the name of the service that runs Chassis

In [10]:
res = chassisml.publish(
    image_data=image_data,
    deploy=True,
    base_url='http://localhost:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Publishing container... Ok!
Job ID: chassis-builder-job-60111259-e475-4c0c-a4a5-10951878bcfb


After the request is made, Chassis launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

This is an example of the data that is shown when the job has not finished yet.

In [11]:
chassisml.get_job_status(job_id)

{'active': 1,
 'completion_time': None,
 'conditions': None,
 'failed': None,
 'start_time': 'Fri, 09 Jul 2021 09:01:43 GMT',
 'succeeded': None}

And this is an example of the data that is shown when the job has already finished.

In [12]:
chassisml.get_job_status(job_id)

{'active': None,
 'completion_time': 'Fri, 09 Jul 2021 09:13:37 GMT',
 'conditions': [{'last_probe_time': 'Fri, 09 Jul 2021 09:13:37 GMT',
   'last_transition_time': 'Fri, 09 Jul 2021 09:13:37 GMT',
   'message': None,
   'reason': None,
   'status': 'True',
   'type': 'Complete'}],
 'failed': None,
 'start_time': 'Fri, 09 Jul 2021 09:01:43 GMT',
 'succeeded': 1}

### Pull the docker image

Now that the job has finished, we can pull and load the docker image that has been generated.

In [13]:
!docker pull XXXX/chassisml-sklearn-demo:latest

latest: Pulling from XXXX/chassisml-sklearn-demo

81a07f80: Already exists 
0ae6b0ab: Pulling fs layer 
57e3f39c: Pulling fs layer 
4ba4cd3e: Pulling fs layer 
14849d82: Pulling fs layer 
14849d82: Waiting fs layer 
fb0632a0: Pulling fs layer 
66153fff: Waiting fs layer 
f33e2f48: Pulling fs layer 
ff3e1f88: Pull complete .11kB/4.11kBBBADownloading    476MB/765MBDigest: sha256:e0b7a5ccb8b1d01d7b467f1173a1c4059a49356a14f491abc9556f56ff312f26
Status: Downloaded newer image for XXXX/chassisml-sklearn-demo:latest
docker.io/XXXX/chassisml-sklearn-demo:latest


In [14]:
!docker images XXXX/chassisml-sklearn-demo:latest

REPOSITORY                          TAG       IMAGE ID       CREATED         SIZE
XXXX/chassisml-sklearn-demo   latest    0e5c5815f2ec   3 minutes ago   2.19GB
